In [59]:
from sklearn.model_selection import TimeSeriesSplit
import numpy as np
import lightgbm as lgb
import pandas as pd

train = pd.read_csv('train.csv')
df_train = pd.read_csv('df_train.csv', index_col='row_id')
df_test = pd.read_csv('df_test.csv', index_col='row_id')
target_train = pd.read_csv('target_train.csv', index_col='row_id').squeeze()
target_test = pd.read_csv('target_test.csv', index_col='row_id').squeeze()

In [ ]:
def smape_loss(y_true, y_pred):
    """
    Custom SMAPE loss function for LightGBM
    """
    return np.mean(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred))) * 100

def smape_cv(model, df_train, target_train):
    tscv = TimeSeriesSplit(n_splits=5)
    smape_list = []
    for _, (train_index, test_index) in enumerate(tscv.split(df_train), start=1):
        # Create training and validation datasets
        X_train = df_train.iloc[train_index]
        y_train = target_train.iloc[train_index]
        X_valid = df_train.iloc[test_index]
        y_valid = target_train.iloc[test_index]

        # Train the model
        model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], eval_metric='mape', verbose=False)

        # Make predictions
        y_pred = np.exp(model.predict(X_valid)) * X_valid['microbusiness_density']
        y_valid = np.exp(y_valid) * X_valid['microbusiness_density']

        # Calculate SMAPE score
        smape = smape_loss(y_valid, y_pred)
        smape_list.append(smape)
    
    print(f'Mean SMAPE: {np.mean(smape_list):.4f}')
    return np.mean(smape_list)

def lgbm_objective(trial, df_train, target_train):
    """
    Objective function for LightGBM optimization without rolling window cross-validation.
    """
    params = {
        'n_estimators'      : 200,
        'verbose'           : -1,
        'objective'         : 'regression_l1',
        'random_state'      : 42,
        'extra_trees'       : True,
        'colsample_bytree'  : trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode'  : trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth'         : trial.suggest_int('max_depth', 3, 10),
        'learning_rate'     : trial.suggest_float('learning_rate', 0.01, 0.1, log=True),
        'lambda_l1'         : trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2'         : trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves'        : trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf'  : trial.suggest_int('min_data_in_leaf', 5, 250),
        'early_stopping_round': 50,}

    model = lgb.LGBMRegressor(**params)
    score = smape_cv(model, df_train, target_train)
    return score

In [15]:
import optuna
import pandas as pd

study = optuna.create_study(direction='minimize')
study.optimize(lambda trial: lgbm_objective(trial, df_train, target_train), n_trials=20)

# Print the best trial
print(f"Best trial: {study.best_trial.value} with parameters {study.best_trial.params}")

[I 2023-04-14 21:10:14,202] A new study created in memory with name: no-name-a1c265f8-a19c-4d54-bb6b-9315034f9c9d
/opt/homebrew/Caskroom/miniforge/base/envs/base_env/lib/python3.8/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=248, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=248
[LightGBM] [Warning] lambda_l2 is set=5.026156487310038, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.026156487310038
[LightGBM] [Warning] lambda_l1 is set=0.6018665058271135, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6018665058271135
[LightGBM] [Warning] min_data_in_leaf is set=248, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=248
[LightGBM] [Warning] lambda_l2 is set=5.026156487310038, reg_lambda=0.0 will be ignored. Current value: lambda_l2=5.026156487310038
[LightGBM] [Warning] lambda_l1 is set=0.6018665058271135, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.6018665058271135
[LightGBM] [Warning] min_data_in_leaf is set=248, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=248
[LightGBM] [Warning] lambda_l2 is set=5.026156487310038, reg_lambda=0.0 will be ignored. Current v

[I 2023-04-14 21:10:23,761] Trial 0 finished with value: 1.7542303345776638 and parameters: {'colsample_bytree': 0.6954366493504512, 'colsample_bynode': 0.888367794528408, 'max_depth': 5, 'learning_rate': 0.05557089796308, 'lambda_l1': 0.6018665058271135, 'lambda_l2': 5.026156487310038, 'num_leaves': 139, 'min_data_in_leaf': 248}. Best is trial 0 with value: 1.7542303345776638.


Mean SMAPE: 1.7542
[LightGBM] [Warning] min_data_in_leaf is set=102, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=102
[LightGBM] [Warning] lambda_l2 is set=9.995542437705659, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.995542437705659
[LightGBM] [Warning] lambda_l1 is set=2.2653315282894138, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.2653315282894138
[LightGBM] [Warning] min_data_in_leaf is set=102, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=102
[LightGBM] [Warning] lambda_l2 is set=9.995542437705659, reg_lambda=0.0 will be ignored. Current value: lambda_l2=9.995542437705659
[LightGBM] [Warning] lambda_l1 is set=2.2653315282894138, reg_alpha=0.0 will be ignored. Current value: lambda_l1=2.2653315282894138
[LightGBM] [Warning] min_data_in_leaf is set=102, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=102
[LightGBM] [Warning] lambda_l2 is set=9.995542437705659, reg_lambda=0.0 will be

[I 2023-04-14 21:10:31,604] Trial 1 finished with value: 1.7753899123422638 and parameters: {'colsample_bytree': 0.6406399870245952, 'colsample_bynode': 0.8551289858335458, 'max_depth': 4, 'learning_rate': 0.010465074144003937, 'lambda_l1': 2.2653315282894138, 'lambda_l2': 9.995542437705659, 'num_leaves': 402, 'min_data_in_leaf': 102}. Best is trial 0 with value: 1.7542303345776638.


...

Mean SMAPE: 1.7515
[LightGBM] [Warning] min_data_in_leaf is set=178, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=178
[LightGBM] [Warning] lambda_l2 is set=6.105983280243719, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.105983280243719
[LightGBM] [Warning] lambda_l1 is set=1.5090908556438065, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5090908556438065
[LightGBM] [Warning] min_data_in_leaf is set=178, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=178
[LightGBM] [Warning] lambda_l2 is set=6.105983280243719, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.105983280243719
[LightGBM] [Warning] lambda_l1 is set=1.5090908556438065, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5090908556438065
[LightGBM] [Warning] min_data_in_leaf is set=178, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=178
[LightGBM] [Warning] lambda_l2 is set=6.105983280243719, reg_lambda=0.0 will be

[I 2023-04-14 21:14:16,740] Trial 18 finished with value: 1.750562904110117 and parameters: {'colsample_bytree': 0.9851161038051562, 'colsample_bynode': 0.6647720786039399, 'max_depth': 10, 'learning_rate': 0.04242766504090565, 'lambda_l1': 1.5090908556438065, 'lambda_l2': 6.105983280243719, 'num_leaves': 687, 'min_data_in_leaf': 178}. Best is trial 18 with value: 1.750562904110117.


Mean SMAPE: 1.7506
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGBM] [Warning] lambda_l2 is set=7.2261788423304285, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.2261788423304285
[LightGBM] [Warning] lambda_l1 is set=3.5490622332470467, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5490622332470467
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGBM] [Warning] lambda_l2 is set=7.2261788423304285, reg_lambda=0.0 will be ignored. Current value: lambda_l2=7.2261788423304285
[LightGBM] [Warning] lambda_l1 is set=3.5490622332470467, reg_alpha=0.0 will be ignored. Current value: lambda_l1=3.5490622332470467
[LightGBM] [Warning] min_data_in_leaf is set=174, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=174
[LightGBM] [Warning] lambda_l2 is set=7.2261788423304285, reg_lambda=0.0 wi

[I 2023-04-14 21:14:35,498] Trial 19 finished with value: 1.7541432597372073 and parameters: {'colsample_bytree': 0.4651571259079887, 'colsample_bynode': 0.7325173422461719, 'max_depth': 10, 'learning_rate': 0.04105100171960733, 'lambda_l1': 3.5490622332470467, 'lambda_l2': 7.2261788423304285, 'num_leaves': 694, 'min_data_in_leaf': 174}. Best is trial 18 with value: 1.750562904110117.


Mean SMAPE: 1.7541
Best trial: 1.750562904110117 with parameters {'colsample_bytree': 0.9851161038051562, 'colsample_bynode': 0.6647720786039399, 'max_depth': 10, 'learning_rate': 0.04242766504090565, 'lambda_l1': 1.5090908556438065, 'lambda_l2': 6.105983280243719, 'num_leaves': 687, 'min_data_in_leaf': 178}


In [31]:
params = study.best_trial.params
LGBM = lgb.LGBMRegressor(**params)

In [33]:
LGBM.fit(df_train, target_train)
y_pred = np.exp(LGBM.predict(df_test)) * df_test.microbusiness_density

[LightGBM] [Warning] min_data_in_leaf is set=178, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=178
[LightGBM] [Warning] lambda_l1 is set=1.5090908556438065, reg_alpha=0.0 will be ignored. Current value: lambda_l1=1.5090908556438065
[LightGBM] [Warning] lambda_l2 is set=6.105983280243719, reg_lambda=0.0 will be ignored. Current value: lambda_l2=6.105983280243719


array([3.36006323, 8.54059812, 1.19667252, ..., 3.91537597, 3.13450942,
       1.81122218])

In [67]:
my_cfips = df_train['cfips'].unique()
train = train[train.cfips.isin(my_cfips)]
pred = y_pred.values
y_true = train[train.row_id.str.contains('2022-07-01')].microbusiness_density.values
print(smape_loss(y_true, pred))

2.1456129466724616


In [ ]:
# clip_err = True

# if clip_err:
#     df_record = y_pred.merge(y_base, on = 'row_id', how='inner')
#     df_record = df_record.merge(y_true, on = 'row_id', how='inner')
#     def smape_(y_true, y_pred):
#         return 100  * np.sum(2 * np.abs(y_pred - y_true) / (np.abs(y_true) + np.abs(y_pred)))
    
#     base_err = df_record.apply(lambda x: smape_(x.y_true, x.y_base), axis=1)
#     pred_err = df_record.apply(lambda x: smape_(x.y_true, x.microbusiness_density), axis=1)
#     blacklist = base_err[(base_err + 1e-3) < pred_err].index
#     y_pred.iloc[blacklist] = y_base.iloc[blacklist]
#     avg_base_err = base_err.mean()
#     avg_pred_err = pred_err.mean()
#     print(avg_base_err, avg_pred_err)
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')
# else:
#     df_sub = pd.read_csv('sample_submission.csv')
#     df_sub = df_sub.drop(columns='microbusiness_density')
#     df_sub['cfips'] = df_sub.row_id.str.split('_', expand=True)[0]
#     y_pred['cfips'] = y_pred.row_id.str.split('_', expand=True)[0]
#     merged_df = df_sub.merge(y_pred, on='cfips', how='left', suffixes=('', '_drop'))
#     merged_df = merged_df.loc[:,~merged_df.columns.str.endswith('_drop')]
#     # merged_df = merged_df.drop(columns='cfips')